In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.ensemble import *
from datetime import datetime
from tensorflow.keras.callbacks import *

In [2]:
df = pd.read_csv("./Datasets/GW9/soccer-spi/spi_matches.csv")

In [3]:
df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def dateconv (dates):
  date = datetime.strptime(dates,'%Y-%m-%d') 
  return date

df['date'] = df['date'].apply(lambda x : dateconv(x))

In [5]:
last_played = datetime(2020,11,20)
GW_end = datetime(2020,11,25)
pl = df[df['league']== "Barclays Premier League"]
old = pl[pl['date']<last_played]
new = pl[pl['date']>last_played]
new = new[new['date'] < GW_end]

In [6]:
teams = pd.get_dummies(new[['team1','team2']])
teamsold = pd.get_dummies(old[['team1','team2']])

In [7]:
new = pd.concat([new,teams], axis=1)
old = pd.concat([old,teamsold], axis=1)

In [8]:
new.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2', 'team1_Aston Villa',
       'team1_Burnley', 'team1_Fulham', 'team1_Leeds United',
       'team1_Liverpool', 'team1_Manchester United', 'team1_Newcastle',
       'team1_Sheffield United', 'team1_Tottenham Hotspur',
       'team1_Wolverhampton', 'team2_Arsenal',
       'team2_Brighton and Hove Albion', 'team2_Chelsea',
       'team2_Crystal Palace', 'team2_Everton', 'team2_Leicester City',
       'team2_Manchester City', 'team2_Southampton',
       'team2_West Bromwich Albion', 'team2_West Ham United'],
      dtype='object')

In [9]:
old_teams = old.columns.tolist()
new_teams = new.columns.tolist()

In [10]:
dropped_columns_old = []
dropped_columns_new = []
for column in old_teams:
    if column not in new_teams:
        dropped_columns_old.append(column)
for column in new_teams:
    if column not in old_teams:
        dropped_columns_new.append(column)

In [11]:
dropped_columns_new

[]

In [9]:
X_train = old[['season', 'spi1', 'spi2', 'score1','score2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2', 'team1_Aston Villa',
       'team1_Burnley', 'team1_Fulham', 'team1_Leeds United',
       'team1_Liverpool', 'team1_Manchester United', 'team1_Newcastle',
       'team1_Sheffield United', 'team1_Tottenham Hotspur',
       'team1_Wolverhampton', 'team2_Arsenal',
       'team2_Brighton and Hove Albion', 'team2_Chelsea',
       'team2_Crystal Palace', 'team2_Everton', 'team2_Leicester City',
       'team2_Manchester City', 'team2_Southampton',
       'team2_West Bromwich Albion', 'team2_West Ham United']]
X_test = new[['season', 'spi1', 'spi2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2','team1_Aston Villa',
       'team1_Burnley', 'team1_Fulham', 'team1_Leeds United',
       'team1_Liverpool', 'team1_Manchester United', 'team1_Newcastle',
       'team1_Sheffield United', 'team1_Tottenham Hotspur',
       'team1_Wolverhampton', 'team2_Arsenal',
       'team2_Brighton and Hove Albion', 'team2_Chelsea',
       'team2_Crystal Palace', 'team2_Everton', 'team2_Leicester City',
       'team2_Manchester City', 'team2_Southampton',
       'team2_West Bromwich Albion', 'team2_West Ham United']]

In [10]:
X_train.dropna(inplace=True)

In [11]:
X_test.dropna(inplace=True)

In [13]:
lm = LinearRegression()
lm.fit(X_train.drop(['score1','score2'],axis=1),X_train[['score1','score2']])
predictions  = lm.predict(X_test)
preds = []
for num in predictions:
  score1 = np.round(num[0])
  score2 = np.round(num[1])
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsPLGW9_LR.xlsx")

10
10


In [14]:
tX_train = X_train.drop(['score1','score2'],axis=1).values
tX_test =X_test.values
y_train = X_train[['score1','score2']].values

In [15]:
scaler = MinMaxScaler()
tX_train = scaler.fit_transform(tX_train)
tX_test = scaler.transform(tX_test)
tX_train.shape

(1598, 28)

In [16]:
def two_to_three(array):
    array = np.reshape(array, array.shape+(1,))
    return array

In [17]:
tX_test = two_to_three(tX_test)
tX_train = two_to_three(tX_train)
print(tX_test.shape)
print(tX_train.shape)

(10, 28, 1)
(1598, 28, 1)


In [18]:
model = Sequential()

In [19]:

model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46))
model.add(Dense(92, activation='relu'))
model.add(Dense(2))


In [20]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [21]:
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=25)

In [22]:
model.fit(tX_train,y_train,epochs=60)

Train on 1598 samples
Epoch 1/60
1598/1598 [==============================] - 4s 3ms/sample - loss: 1.5786 - accuracy: 0.6589
Epoch 2/60
1598/1598 [==============================] - 2s 1ms/sample - loss: 1.3739 - accuracy: 0.6877
Epoch 3/60
1598/1598 [==============================] - 1s 853us/sample - loss: 1.3573 - accuracy: 0.7196
Epoch 4/60
1598/1598 [==============================] - 1s 839us/sample - loss: 1.3421 - accuracy: 0.7009
Epoch 5/60
1598/1598 [==============================] - 2s 942us/sample - loss: 1.3484 - accuracy: 0.7084
Epoch 6/60
1598/1598 [==============================] - 2s 1ms/sample - loss: 1.3415 - accuracy: 0.7153
Epoch 7/60
1598/1598 [==============================] - 2s 1ms/sample - loss: 1.3424 - accuracy: 0.7209
Epoch 8/60
1598/1598 [==============================] - 2s 1ms/sample - loss: 1.3305 - accuracy: 0.7153
Epoch 9/60
1598/1598 [==============================] - 2s 2ms/sample - loss: 1.3056 - accuracy: 0.7103
Epoch 10/60
1598/1598 [=============

In [23]:
predictions  = model.predict(tX_test)
preds = []
for num in predictions:
  score1 = np.round(num[0]) if num[0]>=0 else 0
  score2 = np.round(num[1]) if num[1]>=0 else 0
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsPLGW9_TF.xlsx")

10
10
